In [17]:
scripted_model = torch.jit.script(model)
torch.jit.save(scripted_model, 'model.pt')

In [2]:
import torch
models = []
for _ in range(4):
    models.append(torch.jit.load('MultiSeqUmapEmb_epoch-100_pred_step-6_model.pt'))

In [4]:
from utils import *
r_seq = db2Rseq()
h_seq = db2Hseq(h_step = 4)
t = dt2T()
s_attrs, _ = db2S()

pred_steps = [1, 2, 3, 6]

# model inference
for i, m in enumerate(pred_steps):
    globals()[f'res_{m}'] = torch.exp(models[i](r_seq.float(), h_seq[:, i, :, :].float(), t[:, i, :].int(), s_attrs.float()))

In [5]:
res_1

tensor([[0.1278, 0.2522, 0.6200],
        [0.1248, 0.6716, 0.2036],
        [0.1840, 0.5177, 0.2982],
        ...,
        [1.0000, 0.0000, 0.0000],
        [0.4062, 0.3293, 0.2644],
        [0.9779, 0.0072, 0.0149]], grad_fn=<ExpBackward0>)